In [10]:

!pip install openvino==2022.03

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.5/36.5 MB 3.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [11]:
!python -c "from openvino.runtime import Core"
from openvino.runtime import Core
ie_core=Core()

In [12]:
devices = ie_core.available_devices
print(devices)

['CPU']


In [13]:
onnx_kp = "/home/aistudio/onnx_model/paddlegan_fder.onnx"
model_kp = ie_core.read_model(model=onnx_kp)
kp_net=ie_core.compile_model(model=model_kp,device_name= "AUTO")
output_kpjacobian = kp_net.output(0) #jacobian
output_kpvalue = kp_net.output(1)#value
x = iter(kp_net.inputs)
for i in x:
    print(i)
y = iter(kp_net.outputs)
for i in y:
    print('输出'+str(i))



<ConstOutput: names[eager_tmp_0] shape[1,3,256,256] type: f32>
输出<ConstOutput: names[reshape2_6.tmp_0] shape[1,10,2,2] type: f32>
输出<ConstOutput: names[sum_0.tmp_0] shape[1,10,2] type: f32>


In [14]:
#RuntimeError: AUTOload all devices failed
generator_kp = "/home/aistudio/onnx_model/generator_fder.onnx"
model_generator = ie_core.read_model(model=generator_kp)
generator_net=ie_core.compile_model(model=model_generator,device_name= "AUTO")


RuntimeError: AUTOload all devices failed

In [9]:
#RuntimeError: There should be only one instance of RegistersPool per thread
generator_net=ie_core.compile_model(model=model_generator,device_name= "CPU")

RuntimeError: There should be only one instance of RegistersPool per thread

In [10]:
#Please, check your environment
# Check 'error_code == 0' failed at src/plugins/intel_gpu/src/runtime/ocl/ocl_device_detector.cpp:194:
# [GPU] No supported OCL devices found or unexpected error happened during devices query.
# [GPU] Please check OpenVINO documentation for GPU drivers setup guide.
# [GPU] clGetPlatformIDs error code: -1001
generator_net=ie_core.compile_model(model=model_generator,device_name= "GPU")

RuntimeError: Failed to create plugin /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/openvino/libs/libopenvino_intel_gpu_plugin.so for device GPU
Please, check your environment
Check 'error_code == 0' failed at src/plugins/intel_gpu/src/runtime/ocl/ocl_device_detector.cpp:194:
[GPU] No supported OCL devices found or unexpected error happened during devices query.
[GPU] Please check OpenVINO documentation for GPU drivers setup guide.
[GPU] clGetPlatformIDs error code: -1001



In [ ]:
#onnx_model/generator_fder.onnx,This model can only be deployed on Intel GPUs
output_generator =generator_net.output(0)
x = iter(generator_net.inputs)
for i in x:
    print(i)
y = iter(generator_net.outputs)
for i in y:
    print('输出'+str(i))

In [13]:
#数据处理类函数
from enum import Enum
# from .utils import *
import sys
class LandmarksType(Enum):
    _2D = 1
    _2halfD = 2
    _3D = 3
class NetworkSize(Enum):
    # TINY = 1
    # SMALL = 2
    # MEDIUM = 3
    LARGE = 4

    def __new__(cls, value):
        member = object.__new__(cls)
        member._value_ = value
        return member

    def __int__(self):
        return self.value


class FaceAlignment:
    def __init__(self,
                 landmarks_type,
                 network_size=NetworkSize.LARGE,
                 flip_input=False,
                 face_detector='sfd',
                 verbose=False):
        self.flip_input = flip_input
        self.landmarks_type = landmarks_type
        self.verbose = verbose

        network_size = int(network_size)

        # Get the face detector
        face_detector_module = __import__(
            'ppgan.faceutils.face_detection.detection.' + face_detector,
            globals(), locals(), [face_detector], 0)
        self.face_detector = face_detector_module.FaceDetector(verbose=verbose)

    def get_detections_for_batch(self, images):
        images = images[..., ::-1]
        detected_faces = self.face_detector.detect_from_batch(images.copy())
        results = []

        for i, d in enumerate(detected_faces):
            if len(d) == 0:
                results.append(None)
                continue
            d = d[0]
            d = np.clip(d, 0, None)

            x1, y1, x2, y2 = map(int, d[:4])
            results.append((x1, y1, x2, y2))

        return results

    def get_detections_for_image(self, images):
        images = images[..., ::-1]
        detected_faces = self.face_detector.detect_from_batch(images.copy())
        results = []

        for i, d in enumerate(detected_faces[0]):
            if len(d) == 0:
                results.append(None)
                continue
            d = np.clip(d, 0, None)

            x1, y1, x2, y2 = map(int, d[:-1])
            results.append((x1, y1, x2, y2))

        return results


In [17]:
import numpy as np
from scipy.spatial import ConvexHull

import paddle


def animate_normalize_kp(kp_source,
                 kp_driving,
                 kp_driving_initial,
                 adapt_movement_scale=False,
                 use_relative_movement=False,
                 use_relative_jacobian=False):
    if adapt_movement_scale:
        source_area = ConvexHull(kp_source[output_kpvalue][0].numpy()).volume
        driving_area = ConvexHull(kp_driving_initial[output_kpvalue][0].numpy()).volume
        adapt_movement_scale = np.sqrt(source_area) / np.sqrt(driving_area)
    else:
        adapt_movement_scale = 1

    kp_new = {k: v for k, v in kp_driving.items()}

    if use_relative_movement:
        kp_value_diff = (kp_driving[output_kpvalue] - kp_driving_initial[output_kpvalue])
        kp_value_diff *= adapt_movement_scale
        kp_new['value'] = kp_value_diff + kp_source[output_kpvalue]

        if use_relative_jacobian:
            jacobian_diff = paddle.matmul(
                kp_driving[output_kpjacobian],
                np.matrix(kp_driving_initial[output_kpjacobian]))#paddle.inverse
            kp_new['jacobian'] = paddle.matmul(jacobian_diff,
                                               kp_source[output_kpjacobian])

    return kp_new


In [18]:
import os
import sys
import cv2
import math
import yaml
import pickle
import imageio
import numpy as np
from tqdm import tqdm
import paddle
adapt_scale=False,
face_detector='sfd'
# /home/aistudio/work/配乐10s.mov -i /home/aistudio/work/result.mp4
face_path='/home/aistudio/work/src.jpg'
driving_path='/home/aistudio/work/2.MP4'

def smake_animation(source_image,driving_video,relative=True,adapt_movement_scale=True):

    batch_size=1
    predictions = []
    source = paddle.to_tensor(source_image[np.newaxis].astype(
        np.float32)).transpose([0, 3, 1, 2])

    driving_video_np = np.array(driving_video).astype(np.float32)
    driving_n, driving_h, driving_w, driving_c = driving_video_np.shape

    driving_slices = []
    # whole driving as a single slice
    driving = paddle.to_tensor(
        np.array(driving_video).astype(np.float32)).transpose(
            [0, 3, 1, 2])
    frame_count_in_slice = driving_n
    driving_slices.append(driving)
    #加模型

    # kp_source = kp_detector(source)
    # kp_driving_initial = kp_detector(driving_slices[0][0:1])
    kp_source = kp_net(source)

    kp_driving_initial = kp_net(driving_slices[0][0:1])
    #加模型

    kp_source_batch = {}
    # from IPython.core.debugger import Pdb; Pdb().set_trace()
    kp_source_batch["value"] = paddle.tile(
        paddle.to_tensor(kp_source[output_kpvalue],dtype='float32'), repeat_times=[batch_size, 1, 1])
    kp_source_batch["jacobian"] = paddle.tile(
         paddle.to_tensor(kp_source[output_kpjacobian],dtype='float32'), repeat_times=[batch_size, 1, 1, 1])
    source = paddle.tile(source,
                            repeat_times=[batch_size, 1, 1, 1])
    begin_idx = 0
    for frame_idx in tqdm(
            range(int(np.ceil(float(driving_n) / batch_size)))):
        frame_num = min(batch_size, driving_n - begin_idx)
        slice_id = int(frame_idx * batch_size /
                        frame_count_in_slice)

        internal_start = frame_idx - slice_id * frame_count_in_slice
        internal_end = frame_idx - slice_id * frame_count_in_slice + frame_num

        driving_frame = driving_slices[slice_id][
            internal_start:internal_end]
        #  #加模型 kp_source[output_kp]
        # kp_driving = kp_detector(driving_frame)
        kp_driving = kp_net(driving_frame)
            #加模型

        kp_source_img = {}
        kp_source_img["value"] = kp_source_batch["value"][0:frame_num]
        kp_source_img["jacobian"] = kp_source_batch["jacobian"][
            0:frame_num]

        kp_norm = animate_normalize_kp(
            kp_source=kp_source,
            kp_driving= kp_driving,
            kp_driving_initial= paddle.to_tensor(kp_driving_initial[output_kpjacobian],dtype='float32'),
            use_relative_movement=relative,
            use_relative_jacobian=relative,
            adapt_movement_scale=adapt_movement_scale)
        inputsorce=(source[0:frame_num],kp_source_img,kp_norm)
        # out = generator(source[0:frame_num],
        #                 kp_source=kp_source_img,
        #                 kp_driving=kp_norm)
        out = generator_net(inputsorce)[output_generator]
        img = np.transpose(out['prediction'].numpy(),
                            [0, 2, 3, 1]) * 255.0

        predictions.append(img)
        begin_idx += frame_num
    return np.concatenate(predictions)

def get_prediction(face_image,driving_video):
        # driving_forward = driving_video[i:]
        # driving_backward = driving_video[:(i + 1)][::-1]
        predictions =smake_animation(
            face_image,
            driving_video,
            relative=True,
            adapt_movement_scale=False)
        return predictions
def read_img(path):
    img = imageio.imread(path)
    if img.ndim == 2:
        img = np.expand_dims(img, axis=2)
    # som images have 4 channels
    if img.shape[2] > 3:
        img = img[:, :, :3]
    return img  
def extract_bbox(image):
    detector = FaceAlignment(
        LandmarksType._2D,
        flip_input=False,
        face_detector=face_detector)

    frame = [image]
    predictions = detector.get_detections_for_image(np.array(frame))
    person_num = len(predictions)
    if person_num == 0:
        return np.array([])
    results = []
    face_boxs = []
    h, w, _ = image.shape
    for rect in predictions:
        bh = rect[3] - rect[1]
        bw = rect[2] - rect[0]
        cy = rect[1] + int(bh / 2)
        cx = rect[0] + int(bw / 2)
        margin = max(bh, bw)
        y1 = max(0, cy - margin)
        x1 = max(0, cx - int(0.8 * margin))
        y2 = min(h, cy + margin)
        x2 = min(w, cx + int(0.8 * margin))
        area = (y2 - y1) * (x2 - x1)
        results.append([x1, y1, x2, y2, area])
    # if a person has more than one bbox, keep the largest one
    # maybe greedy will be better?
    sorted(results, key=lambda area: area[4], reverse=True)
    results_box = [results[0]]
    for i in range(1, person_num):
        num = len(results_box)
        add_person = True
        for j in range(num):
            pre_person = results_box[j]
            iou =IOU(pre_person[0], pre_person[1], pre_person[2],
                            pre_person[3], pre_person[4], results[i][0],
                            results[i][1], results[i][2], results[i][3],
                            results[i][4])
            if iou > 0.5:
                add_person = False
                break
        if add_person:
            results_box.append(results[i])
    boxes = np.array(results_box)
    return boxes

def IOU(ax1, ay1, ax2, ay2, sa, bx1, by1, bx2, by2, sb):
    #sa = abs((ax2 - ax1) * (ay2 - ay1))
    #sb = abs((bx2 - bx1) * (by2 - by1))
    x1, y1 = max(ax1, bx1), max(ay1, by1)
    x2, y2 = min(ax2, bx2), min(ay2, by2)
    w = x2 - x1
    h = y2 - y1
    if w < 0 or h < 0:
        return 0.0
    else:
        return 1.0 * w * h / (sa + sb - w * h)


image_size=256
source_image =read_img(face_path)
reader = imageio.get_reader(driving_path)
fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    print("Read driving video error!")
    pass
reader.close()
#from IPython.core.debugger import Pdb; Pdb().set_trace()
driving_video = [
    cv2.resize(frame, (image_size, image_size)) / 255.0
    for frame in driving_video
]    
#切片
results = []
multi_person=False
bboxes =extract_bbox(source_image.copy())
print(str(len(bboxes)) + " persons have been detected")

# for multi person
for rec in bboxes:
    face_image = source_image.copy()[rec[1]:rec[3], rec[0]:rec[2]]
    face_image = cv2.resize(face_image,
                            (image_size, image_size)) / 255.0
    predictions = get_prediction(face_image,driving_video)
    results.append({
        'rec':
        rec,
        'predict':
        [predictions[i] for i in range(predictions.shape[0])]
    })
    if len(bboxes) == 1 or not multi_person:
        break

#---
out_frame = []
ratio=1.0
output='/home/aistudio/output'
filename='result.mp4'
for i in range(len(driving_video)):
    frame = source_image.copy()
    for result in results:
        x1, y1, x2, y2, _ = result['rec']
        h = y2 - y1
        w = x2 - x1
        out = result['predict'][i]
        out = cv2.resize(out.astype(np.uint8), (x2 - x1, y2 - y1))
        if len(results) == 1:
            frame[y1:y2, x1:x2] = out
            break
        else:
            patch = np.zeros(frame.shape).astype('uint8')
            patch[y1:y2, x1:x2] = out
            mask = np.zeros(frame.shape[:2]).astype('uint8')
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            cv2.circle(mask, (cx, cy), math.ceil(h * ratio),
                        (255, 255, 255), -1, 8, 0)
            frame = cv2.copyTo(patch, mask, frame)

    out_frame.append(frame)
imageio.mimsave(os.path.join(output, filename),
                [frame for frame in out_frame],
                fps=fps)



In [1]:
!pip install moviepy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#为生成的视频加上音乐
from moviepy.editor import *

videoclip_1 = VideoFileClip("/home/aistudio/work/2.MP4")
videoclip_2 = VideoFileClip("/home/aistudio/output/result.mp4")

audio_1 = videoclip_1.audio

videoclip_3 = videoclip_2.set_audio(audio_1)
videoclip_3.write_videofile("/home/aistudio/output/finalout.mp4", audio_codec="aac")

Moviepy - Building video /home/aistudio/output/finalout.mp4.
MoviePy - Writing audio in finaloutTEMP_MPY_wvf_snd.mp4
MoviePy - Done.
Moviepy - Writing video /home/aistudio/output/finalout.mp4



Moviepy - Done !
Moviepy - video ready /home/aistudio/output/finalout.mp4


In [2]:
# 导入一些可视化需要的包
import cv2
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 定义一个展示视频的函数
def display(driving, fps, size=(8, 6)):
    fig = plt.figure(figsize=size)

    ims = []
    for i in range(len(driving)):
        cols = []
        cols.append(driving[i])

        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    video = animation.ArtistAnimation(fig, ims, interval=1000.0/fps, repeat_delay=1000)

    plt.close()
    return video

In [4]:
# 展示一下输入的视频, 如果视频太大，时间会非常久，可以跳过这个步骤
video_path = '/home/aistudio/output/finalout.mp4'
video_frames = imageio.mimread(video_path, memtest=False)

# 获得视频的原分辨率
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
HTML(display(video_frames, fps).to_html5_video())

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 